In [89]:
from collections import namedtuple
from enum import Enum

import regex as re


class State(Enum):
    INSIDE = 0
    OUTSIDE = 1
    def flip(self):
        if self == State.INSIDE:
            return State.OUTSIDE
        else:
            return State.INSIDE

def sign(x):
    if x > 0:
        return 1
    else:
        return -1

class Vec(namedtuple("Point", ["y", "x"])):
    def __add__(self, other):
        return Vec(self.y + other.y, self.x + other.x)
    
    def __sub__(self, other):
        return Vec(self.y - other.y, self.x - other.x)
    
    def __mul__(self, other):
        match other:
            case Vec(y, x):
                y_mul, x_mul = y, x
            case int() | float() as scalar:
                y_mul, x_mul = scalar, scalar
            case val:
                raise ValueError(f"Incorrect type of value {val}: {type(val)}")
        return Vec(self.y * y_mul, self.x * x_mul)
    
    def step(self):
        if self.y != 0:
            return Vec(sign(self.y), 0)
        else:
            return Vec(0, sign(self.x))


class Direction(Vec, Enum):
    UP = Vec(-1, 0)
    RIGHT = Vec(0, 1)
    DOWN = Vec(1, 0)
    LEFT = Vec(0, -1)

DIRECTION_LOOKUP = {
    "U": Direction.UP,
    "R": Direction.RIGHT,
    "D": Direction.DOWN,
    "L": Direction.LEFT
}

In [171]:
instruction_regex = r"(.) (\d+) \((.*)\)"
with open("18/example.txt") as f:
    data = f.read().splitlines()
    instructions = []
    for row in data:
        direction, num_steps, color = re.match(instruction_regex, row).groups()
        direction = DIRECTION_LOOKUP[direction]
        num_steps = int(num_steps)
        move = direction * num_steps
        instructions.append((direction, move, color))
instructions

[(<Direction.RIGHT: Vec(y=0, x=1)>, Vec(y=0, x=6), '#70c710'),
 (<Direction.DOWN: Vec(y=1, x=0)>, Vec(y=5, x=0), '#0dc571'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-2), '#5713f0'),
 (<Direction.DOWN: Vec(y=1, x=0)>, Vec(y=2, x=0), '#d2c081'),
 (<Direction.RIGHT: Vec(y=0, x=1)>, Vec(y=0, x=2), '#59c680'),
 (<Direction.DOWN: Vec(y=1, x=0)>, Vec(y=2, x=0), '#411b91'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-5), '#8ceee2'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-2, x=0), '#caa173'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-1), '#1b58a2'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-2, x=0), '#caa171'),
 (<Direction.RIGHT: Vec(y=0, x=1)>, Vec(y=0, x=2), '#7807d2'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-3, x=0), '#a77fa3'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-2), '#015232'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-2, x=0), '#7a21e3')]

In [172]:
from functools import reduce

def calculate_bounds(current_vals: tuple[Vec, Vec, Vec], next_move: Vec) -> tuple[Vec, Vec, Vec]:
    current_position, bounds_min, bounds_max = current_vals
    current_position += next_move
    y_c, x_c = current_position
    bounds_min = Vec(min(y_c, bounds_min.y), min(x_c, bounds_min.x))
    bounds_max = Vec(max(y_c, bounds_max.y), max(x_c, bounds_max.x))

    return current_position, bounds_min, bounds_max

starting_position, min_bounds, max_bounds = reduce(calculate_bounds,
       [instruction[1] for instruction in instructions],
       (Vec(0, 0), Vec(0, 0), Vec(0, 0))
)
starting_position, min_bounds, max_bounds

(Vec(y=0, x=0), Vec(y=0, x=0), Vec(y=9, x=6))

In [173]:
y_min, x_min = min_bounds
move = Vec(abs(y_min) if y_min < 0 else 0, abs(x_min) if x_min < 0 else 0)
starting_position += move
min_bounds += move
max_bounds += move

starting_position, min_bounds, max_bounds

(Vec(y=0, x=0), Vec(y=0, x=0), Vec(y=9, x=6))

In [174]:
max_y, max_x = max_bounds
ground = ["." * (max_x+1)] * (max_y+1)
ground

['.......',
 '.......',
 '.......',
 '.......',
 '.......',
 '.......',
 '.......',
 '.......',
 '.......',
 '.......']

In [175]:
max_y, max_x

(9, 6)

In [176]:
max_y, max_x = max_bounds
ground = ["." * (max_x+1)] * (max_y+1)
ground


['.......',
 '.......',
 '.......',
 '.......',
 '.......',
 '.......',
 '.......',
 '.......',
 '.......',
 '.......']

In [177]:
def dig_initial_tunnel(ground, instructions, starting_position):
    current_position = starting_position
    for _, move, _ in instructions:
        attr = "y" if move.y != 0 else "x"
        current_y, current_x = current_position
            
        move_attr = getattr(move, attr)

        if attr == "x":
            destination = current_x + move_attr
            start, end = (destination, current_x) if destination < current_x else (current_x, destination)

            ground[current_y] = ground[current_y][:start] + "#" * (abs(move_attr)+1) + ground[current_y][end+1:]
            
            current_position += move
        else:
            for i in range(abs(move_attr)):
                current_y, current_x = current_position
                ground[current_y] = ground[current_y][:current_x] + "#" + ground[current_y][current_x + 1:]
                current_position += move.step()
    return ground
ground = dig_initial_tunnel(ground, instructions, starting_position)
ground

['#######',
 '#.....#',
 '###...#',
 '..#...#',
 '..#...#',
 '###.###',
 '#...#..',
 '##..###',
 '.#....#',
 '.######']

In [178]:
from tqdm import tqdm

class Corner(Enum):
    UPPER_LEFT = 0
    UPPER_RIGHT = 1
    LOWER_LEFT = 2
    LOWER_RIGHT = 3

transitions = {
    (Corner.UPPER_LEFT, Corner.UPPER_RIGHT): False,
    (Corner.UPPER_LEFT, Corner.LOWER_RIGHT): True,
    (Corner.LOWER_LEFT, Corner.LOWER_RIGHT): False,
    (Corner.LOWER_LEFT, Corner.UPPER_RIGHT): True
}

def check_corner(ground: list[str], y: int, x: int) -> Corner | None:
    row = ground[y]
    if row[x] != "#":
        return None

    if y < (len(ground) - 1) and x < (len(row) - 1) and row[x+1] == "#" and ground[y+1][x] == "#":
        return Corner.UPPER_LEFT
    

    if y < (len(ground) - 1) and x > 0 and row[x-1] == "#" and ground[y+1][x] == "#":
        return Corner.UPPER_RIGHT
    

    if y > 0 and x < (len(row) - 1) and row[x+1] == "#" and ground[y-1][x] == "#":
        return Corner.LOWER_LEFT

    if y > 0 and x > 0 and row[x-1] == "#" and ground[y-1][x] == "#":
        return Corner.LOWER_RIGHT

    

def dig(ground: list[str]):
    rows = []
    # pbar = tqdm(total=len(ground) * len(ground[0]))
    pbar = tqdm(total=len(ground))

    for y in range(len(ground)):
        pbar.update(1)
        state = State.OUTSIDE
        x = 0
        res = ""
        row = ground[y]

        def handle_corners(corner, y, x, res):
            next_hash_x = row.find("#.", x+1)
            if next_hash_x == -1:
                res += row[x:]
                should_continue = False
                should_transition = False
                next_x = -1
                return res, should_continue, should_transition, next_x
            else:
                next_corner = check_corner(ground, y, next_hash_x)
                if next_corner is None:
                    raise ValueError("Unhaled 1")
                
                should_transition = transitions[(corner, next_corner)] # type: ignore
                res += row[x:next_hash_x+1]
                next_x = next_hash_x + 1
                should_continue = True
                return res, should_continue, should_transition, next_x
                
        
        while x < len(row):
            char = row[x]

            if x == (len(row) - 1):
                if state == State.INSIDE and char == ".":
                    res += "#"
                else:
                    res += char
                break

            if state == State.OUTSIDE:
                if char == "#":
                    corner = check_corner(ground, y, x)
                    if corner:
                        res, should_continue, should_transition, next_x = handle_corners(corner, y, x, res)
                        if not should_continue:
                            break

                        if should_transition:
                            state = state.flip()
                        x = next_x
                        # next_hash_x = row.find("#.", x+1)
                        # if next_hash_x == -1:
                        #     res += row[x:]
                        #     break
                        # else:
                        #     next_corner = check_corner(ground, y, next_hash_x)
                        #     if next_corner is None:
                        #         raise ValueError("Unhaled 1")
                        #     new_state = transitions[(corner, next_corner)] # type: ignore
                        #     res += row[x:next_hash_x+1]
                        #     x = next_hash_x + 1
                        #     state = new_state

                    else:
                        if row[x+1] == ".": # border
                            res += "#"
                            x += 1
                            state = State.INSIDE

                        else:
                            raise ValueError("Unhanled 222")
                
                else:
                    res += "."
                    x += 1
            
            else:
                if char == ".":
                    res += "#"
                    x += 1
                
                else:
                    corner = check_corner(ground, y, x)
                    if corner:
                        res, should_continue, should_transition, next_x = handle_corners(corner, y, x, res)
                        if not should_continue:
                            break

                        if should_transition:
                            state = state.flip()
                        x = next_x
                    
                    else:
                        if row[x+1] == ".": # border
                            res += "#"
                            x += 1
                            state = State.OUTSIDE

                        else:
                            raise ValueError("Unhanled 333")

        
        rows.append(res)
    return rows


In [179]:
dug

['#######',
 '#######',
 '#######',
 '..#####',
 '..#####',
 '#######',
 '#####..',
 '#######',
 '.######',
 '.######']

In [180]:
dug = dig(ground)

100%|██████████| 10/10 [00:00<00:00, 89240.51it/s]


In [181]:
with open("ble.txt", "wt") as f:
    f.write("\n".join(ground))
with open("dug.txt", "wt") as f:
    f.write("\n".join(dug))

In [182]:
"\n".join(dug).count("#")

62

In [183]:
def shape_vertices(instructions, starting_position):
    vertices = []
    current_position = starting_position
    for _, move, _ in instructions:
        vertices.append(current_position)
        current_position += move
    return vertices
vertices = shape_vertices(instructions, starting_position)
vertices

[Vec(y=0, x=0),
 Vec(y=0, x=6),
 Vec(y=5, x=6),
 Vec(y=5, x=4),
 Vec(y=7, x=4),
 Vec(y=7, x=6),
 Vec(y=9, x=6),
 Vec(y=9, x=1),
 Vec(y=7, x=1),
 Vec(y=7, x=0),
 Vec(y=5, x=0),
 Vec(y=5, x=2),
 Vec(y=2, x=2),
 Vec(y=2, x=0)]

In [186]:
def shoelace(vertices: list[Vec]):
    area = 0
    outer_points = 0
    for p1, p2 in zip(vertices, vertices[1:] + [vertices[0]]):
        area += (p1.y + p2.y) * (p1.x-p2.x)
        outer_points += abs(sum(p1 - p2))
    print(area)
    print(outer_points)
    return abs(area) / 2 + outer_points / 2 - 1

In [ ]:
A = I + B/2 - 1
A + 1 = I + B/2
A + B/2 + 1 = I + B
B = 38


In [187]:
shoelace(vertices)

84
38


60.0

## Part 2

In [164]:
instruction_regex = r"(.) (\d+) \(#(.*)\)"
with open("18/input.txt") as f:
    data = f.read().splitlines()
    instructions = []
    for row in data:
        direction, num_steps, color = re.match(instruction_regex, row).groups()
        num_steps = int(color[:5], 16)
        direction = {
            "0": "R",
            "1": "D",
            "2": "L",
            "3": "U"
        }[color[5]]


        
        direction = DIRECTION_LOOKUP[direction]
        move = direction * num_steps
        instructions.append((direction, move, color))
instructions

[(<Direction.RIGHT: Vec(y=0, x=1)>, Vec(y=0, x=21116), '0527c0'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-244572, x=0), '3bb5c3'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-595666), '916d22'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-328874, x=0), '504aa3'),
 (<Direction.RIGHT: Vec(y=0, x=1)>, Vec(y=0, x=595666), '916d20'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-102442, x=0), '1902a3'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-21116), '0527c2'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-109781, x=0), '1acd53'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-146408), '23be82'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-477289, x=0), '748693'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-133162), '2082a2'),
 (<Direction.UP: Vec(y=-1, x=0)>, Vec(y=-278054, x=0), '43e263'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-423080), '674a82'),
 (<Direction.DOWN: Vec(y=1, x=0)>, Vec(y=278054, x=0), '43e261'),
 (<Direction.LEFT: Vec(y=0, x=-1)>, Vec(y=0, x=-331161), '50d992')

In [165]:
from functools import reduce


starting_position, min_bounds, max_bounds = reduce(calculate_bounds,
       [instruction[1] for instruction in instructions],
       (Vec(0, 0), Vec(0, 0), Vec(0, 0))
)
starting_position, min_bounds, max_bounds

(Vec(y=0, x=0), Vec(y=-15974218, x=-4788655), Vec(y=219218, x=12466791))

In [166]:
y_min, x_min = min_bounds
move = Vec(abs(y_min) if y_min < 0 else 0, abs(x_min) if x_min < 0 else 0)
starting_position += move
min_bounds += move
max_bounds += move

starting_position, min_bounds, max_bounds

(Vec(y=15974218, x=4788655), Vec(y=0, x=0), Vec(y=16193436, x=17255446))

In [167]:
vertices = shape_vertices(instructions, starting_position)
vertices

[Vec(y=15974218, x=4788655),
 Vec(y=15974218, x=4809771),
 Vec(y=15729646, x=4809771),
 Vec(y=15729646, x=4214105),
 Vec(y=15400772, x=4214105),
 Vec(y=15400772, x=4809771),
 Vec(y=15298330, x=4809771),
 Vec(y=15298330, x=4788655),
 Vec(y=15188549, x=4788655),
 Vec(y=15188549, x=4642247),
 Vec(y=14711260, x=4642247),
 Vec(y=14711260, x=4509085),
 Vec(y=14433206, x=4509085),
 Vec(y=14433206, x=4086005),
 Vec(y=14711260, x=4086005),
 Vec(y=14711260, x=3754844),
 Vec(y=14560926, x=3754844),
 Vec(y=14560926, x=3524477),
 Vec(y=14349078, x=3524477),
 Vec(y=14349078, x=3356709),
 Vec(y=14137120, x=3356709),
 Vec(y=14137120, x=2944004),
 Vec(y=13741476, x=2944004),
 Vec(y=13741476, x=3524477),
 Vec(y=13614905, x=3524477),
 Vec(y=13614905, x=3378099),
 Vec(y=13543221, x=3378099),
 Vec(y=13543221, x=3256818),
 Vec(y=13641189, x=3256818),
 Vec(y=13641189, x=2746786),
 Vec(y=14084638, x=2746786),
 Vec(y=14084638, x=2252927),
 Vec(y=14496853, x=2252927),
 Vec(y=14496853, x=1765687),
 Vec(y=1438825

In [168]:
shoelace(vertices)

97874103749720.0

In [ ]:
97874103749720